In [1]:
%run model_functions.ipynb

-----------------

Start by exploring academic training dataset.

In [8]:
all_words = pd.read_csv('data/train/newspapers/words.csv', encoding='ISO--8859-1', na_filter=False)
display(len(all_words))
all_words.head()

35740

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,0,Latest,late,AJS,,,0,a1e-fragment01,a1e,newspapers
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,a1e,newspapers
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,a1e,newspapers
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,a1e,newspapers
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,a1e,newspapers


In [16]:
all_words[(all_words['function']!='mrw')
          & (all_words['function']!='')
          & (all_words['function']!='mFlag')]

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre


In [7]:
len(all_words.groupby('text_id'))  # academic

10

In [19]:
len(all_words.groupby('text_id'))  # news

14

In [12]:
len(all_words.groupby('text_id'))  # conversation

20

In [14]:
len(all_words.groupby('text_id'))  # fiction

11

## Keep only content words and filter stop verbs!

In [179]:
# Keeping content words: http://ucrel.lancs.ac.uk/bnc2/bnc2guide.htm#m2adv
# Does not include forms of 'be', 'do', 'have', or modal verbs,
# as suggested in https://github.com/EducationalTestingService/metaphor/tree/master/content-words
content_word_pos = set([
    'NN1', 'NN2', 'NN0',
    'NP0',
    'NN1-NP0', 'NN1-VVB', 'NN1-VVG', 'NN2-VVZ',
    'NP0-NN1',
    'VVB', 'VVD', 'VVG', 'VVI', 'VVN', 'VVZ',
    'VVB-NN1', 'VVD-VVN', 'VVD-AJ0', 'VVG-AJ0', 'VVG-NN1', 'VVZ-NN2',
    'AJ0', 'AJC', 'AJS',
    'AJ0-NN1', 'AJ0-VVG', 'AJ0-VVN',
    'AV0', 'AVQ', 'AVP',
    'AV0-AJ0'
])

In [180]:
content_words = all_words[all_words['word_type'].isin(content_word_pos)]
display(len(content_words))
content_words.head(10)

25360

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic
6,25076,KAHLO,kahlo,NP0,,,1267,a6u-fragment02,a6u,academic
7,25077,CULT,cult,NN1,,,1267,a6u-fragment02,a6u,academic
8,25078,Oriana,oriana,NP0-NN1,,,1268,a6u-fragment02,a6u,academic
9,25079,Baddeley,baddeley,NP0,,,1268,a6u-fragment02,a6u,academic
13,25083,witnessed,witness,VVN,mrw,met,1269,a6u-fragment02,a6u,academic
15,25085,shift,shift,NN1-VVB,,,1269,a6u-fragment02,a6u,academic


## Sparsification

Create a vocabulary using content words. This will be used as the unigram features.

In [181]:
vocabulary = content_words[['word']].drop_duplicates().set_index('word')

display(len(vocabulary))
vocabulary.sample(10)

7327

""
word
too
beforehand
chronic
Kates
Terney
conference
Economic
incorrigible
Jackson


In [182]:
def sparse_featurize(words, vocabulary):
    print('Build iterable sentences.')
    iterable_sentences = words.groupby('sentence_id')['word'].apply(list)
    display(len(iterable_sentences))
    display(iterable_sentences[:5])

    indices = []
    data = []
    indptr = [0]
    for i, word, sentence_id in words[['word', 'sentence_id']].itertuples():
        sentence = iterable_sentences[sentence_id]

        # Used to generate a row of 0's and 1's for sparse matrix
        unique_unigram_indices = set()
        for context_word in sentence:
            if context_word == word:
                continue
            if context_word not in vocabulary.index:
                unique_unigram_indices.add(len(vocabulary))
            unique_unigram_indices.add(vocabulary.index.get_loc(context_word))

        # Data = 1 at column index of sparse matrix = row index of word in vocabulary.
        indices.extend(unique_unigram_indices)  # Indices of the data values in this row.
        data.extend(np.ones(len(unique_unigram_indices), dtype=int))  # Data of all one's.
        indptr.append(indptr[-1]+len(unique_unigram_indices))

        if not i % 2000:
            print('-', end='', flush=True)

    return (
        csr_matrix((data, indices, indptr), dtype=int, shape=(len(words), len(vocabulary)+1)),
        iterable_sentences
    )

Width of sparse matrix is len(vocabulary)+1 to process unknown words.

In [183]:
words = content_words
sparse_unigram_features, iterable_sentences = sparse_featurize(words, vocabulary)

scipy.sparse.save_npz('data/sparse_unigram_features.npz', sparse_unigram_features)
display(sparse_unigram_features)
sparse_unigram_features.shape

Build iterable sentences.


1961

sentence_id
1267    [DRESS, HANGS, HERE', DE-FROCKING, KAHLO, CULT]                                                                               
1268    [Oriana, Baddeley]                                                                                                            
1269    [witnessed, shift, art, establishment, attitudes, art, produced, traditional, parameters]                                     
1270    [work, previously, marginalised, artists, become, area, rich, speculation, art, dealers, priced, modern, masters, market]     
1271    [Almost, year, witnessed, discovery, new, artistic, terrain, graffiti, art, Soviet, art, Australian, art, art, Latin, America]
Name: word, dtype: object

-----------

<25360x7328 sparse matrix of type '<class 'numpy.int32'>'
	with 386398 stored elements in Compressed Sparse Row format>

(25360, 7328)

Inspect if sparse features are valid.

In [184]:
sparse_unigram_features = scipy.sparse.load_npz('data/sparse_unigram_features.npz')

In [185]:
select_word_i = 210

selected_word = words.iloc[select_word_i]
display(selected_word)
sentence_of_selected_word = sorted(list(set(iterable_sentences.loc[selected_word.sentence_id])))
display(sentence_of_selected_word)

unigram_select_indices = np.nonzero(sparse_unigram_features[select_word_i])[:50][1]  # Nonzero indices indicating word.
display(unigram_select_indices)
sparse_matrix_sentence_selections = sorted(list(vocabulary.iloc[unigram_select_indices].index))
display(sparse_matrix_sentence_selections)

assert len(sentence_of_selected_word) - 1 == len(sparse_matrix_sentence_selections)

word_id        25449         
word           passionate    
lemma          passionate    
word_type      AJ0           
function                     
seg_type                     
sentence_id    1282          
text_id        a6u-fragment02
text_tag       a6u           
genre          academic      
Name: 379, dtype: object

['Diego',
 'Rivera',
 'appearance',
 'come',
 'dominate',
 'emotional',
 'flamboyant',
 'husband',
 'obsession',
 'pain',
 'passionate',
 'physical',
 'responses',
 'work']

array([ 70,  71, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,  16])

['Diego',
 'Rivera',
 'appearance',
 'come',
 'dominate',
 'emotional',
 'flamboyant',
 'husband',
 'obsession',
 'pain',
 'physical',
 'responses',
 'work']

Looks good!

Do you wanna build a model?

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.estimator_checks import check_estimator

def preprocess_data(words):
    def build_content_words(all_words):
        print('Building content words.')
        # Keeping content words: http://ucrel.lancs.ac.uk/bnc2/bnc2guide.htm#m2adv
        # Does not include forms of 'be', 'do', 'have', or modal verbs,
        # as suggested in https://github.com/EducationalTestingService/metaphor/tree/master/content-words
        content_word_pos = set([
            'NN1', 'NN2', 'NN0',
            'NP0',
            'NN1-NP0', 'NN1-VVB', 'NN1-VVG', 'NN2-VVZ',
            'NP0-NN1',
            'VVB', 'VVD', 'VVG', 'VVI', 'VVN', 'VVZ',
            'VVB-NN1', 'VVD-VVN', 'VVD-AJ0', 'VVG-AJ0', 'VVG-NN1', 'VVZ-NN2',
            'AJ0', 'AJC', 'AJS',
            'AJ0-NN1', 'AJ0-VVG', 'AJ0-VVN',
            'AV0', 'AVQ', 'AVP',
            'AV0-AJ0'
        ])
        
        content_words = all_words[all_words['word_type'].isin(content_word_pos)]
        
        return content_words
    
    def get_test_fold_ids(words):
        num_folds = {
            'newspapers': 10,
            'conversation': 10,
            'academic': 10,
            'fiction': 11
        }
        
        genre = words.iloc[0]['genre']

        test_fold_ids = defaultdict(int)

        for i, (text, _) in enumerate(words.groupby('text_id')):
            test_fold_ids[text] = i // num_folds[genre]

        return words['text_id'].apply(lambda x: test_fold_ids[x])
        
    
    def build_labels(words):
        labels = words['function'] == 'mrw'
        
        return labels
    
    content_words = build_content_words(words)
    test_fold_ids = get_test_fold_ids(content_words)
    labels = build_labels(content_words)
    
    assert len(content_words) == len(test_fold_ids)
    
    return content_words, test_fold_ids, labels

Model

In [211]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.estimator_checks import check_estimator

def preprocess_data(words):
    def build_content_words(all_words):
        print('Building content words.')
        # Keeping content words: http://ucrel.lancs.ac.uk/bnc2/bnc2guide.htm#m2adv
        # Does not include forms of 'be', 'do', 'have', or modal verbs,
        # as suggested in https://github.com/EducationalTestingService/metaphor/tree/master/content-words
        content_word_pos = set([
            'NN1', 'NN2', 'NN0',
            'NP0',
            'NN1-NP0', 'NN1-VVB', 'NN1-VVG', 'NN2-VVZ',
            'NP0-NN1',
            'VVB', 'VVD', 'VVG', 'VVI', 'VVN', 'VVZ',
            'VVB-NN1', 'VVD-VVN', 'VVD-AJ0', 'VVG-AJ0', 'VVG-NN1', 'VVZ-NN2',
            'AJ0', 'AJC', 'AJS',
            'AJ0-NN1', 'AJ0-VVG', 'AJ0-VVN',
            'AV0', 'AVQ', 'AVP',
            'AV0-AJ0'
        ])
        
        content_words = all_words[all_words['word_type'].isin(content_word_pos)]
        
        return content_words
    
    def get_test_fold_ids(words):
        num_folds = {
            'newspapers': 10,
            'conversation': 10,
            'academic': 10,
            'fiction': 11
        }
        
        genre = words.iloc[0]['genre']

        test_fold_ids = defaultdict(int)

        for i, (text, _) in enumerate(words.groupby('text_id')):
            test_fold_ids[text] = i // num_folds[genre]

        return words['text_id'].apply(lambda x: test_fold_ids[x])
        
    
    def build_labels(words):
        labels = words['function'] == 'mrw'
        
        return labels
    
    content_words = build_content_words(words)
    test_fold_ids = get_test_fold_ids(content_words)
    labels = build_labels(content_words)
    
    assert len(content_words) == len(test_fold_ids)
    
    return content_words, test_fold_ids, labels

class UnigramClassifier(BaseEstimator):
    def __init__(self, verbose=10, random_state=0, C=10, penalty='l1',
                 solver='liblinear', class_weight='balanced', max_iter=500):
        self.verbose = verbose
        self.random_state = random_state
        self.C = C
        self.penalty = penalty
        self.solver = solver
        self.class_weight = class_weight
        self.max_iter = max_iter
 
    
    def fit(self, X, y):
        self.build_features(X, y)
        
        self.clf_ = LogisticRegression(
            verbose=self.verbose,
            random_state=self.random_state,
            C=self.C,
            penalty=self.penalty,
            solver=self.solver,
            class_weight=self.class_weight,
            max_iter=self.max_iter
        )
        
        self.clf_.fit(self.X_, self.y_)
        
        return self
    
    def build_features(self, X, y):
        self.vocabulary_ = X[['word']].drop_duplicates().set_index('word')
        
        sparse_unigram_features, iterable_sentences = self.sparse_featurize(X)
        
        self.X_, self.y_ = shuffle(sparse_unigram_features, y, random_state=0)
    
    
    def sparse_featurize(self, words):
        iterable_sentences = words.groupby('sentence_id')['word'].apply(list)

        indices = []
        data = []
        indptr = [0]
        for i, word, sentence_id in words[['word', 'sentence_id']].itertuples():
            sentence = iterable_sentences[sentence_id]

            # Used to generate a row of 0's and 1's for sparse matrix
            unique_unigram_indices = set()
            for context_word in sentence:
                if context_word == word:
                    continue
                if context_word not in self.vocabulary_.index:
                    unique_unigram_indices.add(len(self.vocabulary_))
                else:
                    unique_unigram_indices.add(self.vocabulary_.index.get_loc(context_word))

            # Data = 1 at column index of sparse matrix = row index of word in vocabulary.
            indices.extend(unique_unigram_indices)  # Indices of the data values in this row.
            data.extend(np.ones(len(unique_unigram_indices), dtype=int))  # Data of all one's.
            indptr.append(indptr[-1]+len(unique_unigram_indices))

            if not i % 2000:
                print('-', end='', flush=True)

        return (
            csr_matrix((data, indices, indptr), dtype=int, shape=(len(words), len(self.vocabulary_)+1)),
            iterable_sentences
        )   
    
    
    def predict(self, X):
        featurized_X = self.sparse_featurize(X)[0]
        return self.clf_.predict(featurized_X)
    
# check_estimator(UnigramClassifier)

Try on academic.

In [304]:
X, test_fold_ids, vocabulary, y = preprocess_data(all_words)

-----------

In [3]:
unigram_classifier = UnigramClassifier(
    verbose=10, random_state=0, C=10, penalty='l1',
    solver='liblinear', class_weight='balanced', max_iter=500,
    vocabulary=vocabulary
)

C = np.logspace(-2,4,20)
penalty = ['l1', 'l2']
hyperparameters = {'C':C, 'penalty':penalty}
clf = GridSearchCV(unigram_classifier, hyperparameters,
                   cv=PredefinedSplit(test_fold_ids),
                   verbose=10, scoring='f1')

NameError: name 'vocabulary' is not defined

In [ ]:
clf.fit(X, y)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[CV] C=0.01, penalty=l1 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 17 epochs took 0 seconds
[CV] ..... C=0.01, penalty=l1, score=0.3702857142857143, total=   0.3s
[CV] C=0.01, penalty=l1 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 18 epochs took 0 seconds
[CV] ..... C=0.01, penalty=l1, score=0.3415032679738562, total=   0.3s
[CV] C=0.01, penalty=l1 ..............................................


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 10 seconds
[CV]  C=14.38449888287663, penalty=l2, score=0.05799256505576208, total=   9.2s
[CV] C=14.38449888287663, penalty=l2 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 19 seconds
[CV]  C=14.38449888287663, penalty=l2, score=0.2438391699092088, total=  19.5s
[CV] C=14.38449888287663, penalty=l2 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [193]:
clf.best_params_, clf.best_estimator_

({'C': 0.04281332398719394, 'penalty': 'l1'},
 UnigramClassifier(C=0.04281332398719394, class_weight='balanced',
          max_iter=500, penalty='l1', random_state=0, solver='liblinear',
          verbose=10))

In [194]:
test_set = pd.read_csv('data/test/academic/words.csv', encoding='ISO--8859-1', na_filter=False)
display(len(test_set))
test_set.head()

17223

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,61516,BRINGING,bring,VVG,mrw,met,3232,amm-fragment02,amm,academic
1,61517,FOSSILS,fossil,NN2,,,3232,amm-fragment02,amm,academic
2,61518,BACK,back,AVP,mrw,met,3232,amm-fragment02,amm,academic
3,61519,TO,to,PRP,mrw,met,3232,amm-fragment02,amm,academic
4,61520,LIFE,life,NN1,mrw,met,3232,amm-fragment02,amm,academic


--------------------

Let's try our model on all 4 genres!

In [3]:
results = {
#     'academic':None,
    'fiction':None,
    'conversation':None,
#     'newspapers':None
}

for genre in results:
    with open('results/U_{}.txt'.format(genre), 'w') as f:
        try:
            train_set = pd.read_csv('data/train/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
            X_train, test_fold_ids, vocabulary, _, y_train = preprocess_data(train_set)
            
            unigram_classifier = UnigramClassifier(
                verbose=10, random_state=0, C=10, penalty='l1',
                solver='liblinear', class_weight='balanced', max_iter=500,
                vocabulary=vocabulary
            )

            C = np.logspace(-2,4,20)
            penalty = ['l1', 'l2']
            hyperparameters = {'C':C, 'penalty':penalty}
            clf = GridSearchCV(unigram_classifier, hyperparameters,
                               cv=PredefinedSplit(test_fold_ids),
                               verbose=10, scoring='f1', n_jobs=-1)
            
            clf.fit(X_train, y_train)

            test_set = pd.read_csv('data/test/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
            X_test, _, _, _, y_test = preprocess_data(test_set, vocabulary)
            preds = clf.predict(X_test)

            result = classification_report(y_test, preds, output_dict=True)
            print(result)
            f.write(json.dumps(result))
            results[genre] = result
        except Exception as e:
            print(e)
            print(results)
            raise e

fiction
-----------(17879, 6033)
(17879, 6033)
Fitting 11 folds for each of 40 candidates, totalling 440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   

[LibLinear]fiction
--(3361, 6033)
(3361, 6033)
{'False': {'precision': 0.8522417153996101, 'recall': 0.7705322523792739, 'f1-score': 0.8093298778230285, 'support': 2837}, 'True': {'precision': 0.18216080402010051, 'recall': 0.2767175572519084, 'f1-score': 0.2196969696969697, 'support': 524}, 'micro avg': {'precision': 0.6935435882177923, 'recall': 0.6935435882177923, 'f1-score': 0.6935435882177923, 'support': 3361}, 'macro avg': {'precision': 0.5172012597098553, 'recall': 0.5236249048155912, 'f1-score': 0.5145134237599991, 'support': 3361}, 'weighted avg': {'precision': 0.7477720939884638, 'recall': 0.6935435882177923, 'f1-score': 0.7174025812273561, 'support': 3361}}
conversation
------(15145, 2874)
(15145, 2874)
Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0458s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.0314s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Batch computation too slow (4.5798s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Batch computation too slow (4.0123s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 226 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done 237 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 250 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 263 ta

[LibLinear]conversation
--(2650, 2874)
(2650, 2874)
{'False': {'precision': 0.9050337311883757, 'recall': 0.7364864864864865, 'f1-score': 0.8121071012805587, 'support': 2368}, 'True': {'precision': 0.13692946058091288, 'recall': 0.35106382978723405, 'f1-score': 0.19701492537313434, 'support': 282}, 'micro avg': {'precision': 0.6954716981132075, 'recall': 0.6954716981132075, 'f1-score': 0.6954716981132075, 'support': 2650}, 'macro avg': {'precision': 0.5209815958846443, 'recall': 0.5437751581368603, 'f1-score': 0.5045610133268466, 'support': 2650}, 'weighted avg': {'precision': 0.8232958427690155, 'recall': 0.6954716981132075, 'f1-score': 0.7466520093538064, 'support': 2650}}


In [60]:
results

{'academic': None, 'newspapers': None}

In [3]:
genre = 'academic'

with open('results/GT_{}.txt'.format(genre), 'w') as f:
    train_set = pd.read_csv('data/train/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
    X_train, test_fold_ids, vocabulary, gt_vocab, y_train = preprocess_data(
        train_set, want_lc=True, want_gt=True)

    unigram_classifier = UnigramClassifier(
        verbose=10, random_state=0, C=10, penalty='l1',
        solver='liblinear', class_weight='balanced', max_iter=500,
        vocabulary=vocabulary
    )

    C = np.logspace(-2,4,20)
    penalty = ['l1', 'l2']
    hyperparameters = {'C':C, 'penalty':penalty}
    clf = GridSearchCV(unigram_classifier, hyperparameters,
                       cv=PredefinedSplit(test_fold_ids),
                       verbose=10, scoring='f1', n_jobs=-1)

    clf.fit(X_train, y_train)

    test_set = pd.read_csv('data/test/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
    X_test, _, _, _, y_test = preprocess_data(
        test_set, vocabulary=vocabulary, gt_vocab=gt_vocab, want_lc=True, want_gt=True)
    preds = clf.predict(X_test)

    result = classification_report(y_test, preds, output_dict=True)
    print(result)
    f.write(json.dumps(result))

academic
-----------(25360, 7328)


155

C:\Users\ezutp\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

...

...

Calculating for paragraph 2304
Calculating for paragraph 2305


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
11208,124192,historical,historical,AJ0,,,6573,cty-fragment03,cty,academic
16643,143950,means,mean,VVZ,,,7706,fef-fragment03,fef,academic
5592,118576,open,open,AJ0,mrw,met,6302,clw-fragment01,clw,academic
12659,125643,proof,proof,NN1,,,6619,cty-fragment03,cty,academic
1434,62950,Vertebra,vertebra,NN1-NP0,,,3297,amm-fragment02,amm,academic


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre,GT
1025,63523,preserved,preserve,VVN,,,3343,amm-fragment02,amm,academic,NaN
3654,120121,make,make,VVI,mrw,met,6377,clw-fragment01,clw,academic,NaN
1635,64746,aspects,aspect,NN2,mrw,met,3408,amm-fragment02,amm,academic,0.187513
792,63100,rods,rod,NN2,mrw,lit,3308,amm-fragment02,amm,academic,NaN
8225,143689,only,only,AV0,,,7691,fef-fragment03,fef,academic,0.291800


Nulls: 8626 845
(8626, 7330)
{'False': {'precision': 0.7738383024442653, 'recall': 0.42719454329774614, 'f1-score': 0.5504920225470527, 'support': 6744}, 'True': {'precision': 0.21211503161329798, 'recall': 0.5526036131774708, 'f1-score': 0.3065585851142225, 'support': 1882}, 'micro avg': {'precision': 0.4545559935079991, 'recall': 0.4545559935079991, 'f1-score': 0.4545559935079991, 'support': 8626}, 'macro avg': {'precision': 0.4929766670287816, 'recall': 0.4898990782376085, 'f1-score': 0.4285253038306376, 'support': 8626}, 'weighted avg': {'precision': 0.6512828658915316, 'recall': 0.4545559935079991, 'f1-score': 0.49727120997476115, 'support': 8626}}


## Other

In [6]:
%run model_functions.ipynb

In [2]:
for genre in ['academic', 'conversation']:
    name = 'LCGTGWCDEPAC'
    do_experiment(
        genre=genre,
        name=name,
        want_lc=True,
        want_gt=True,
        want_gwc=True,
        want_ac=True
    )

academic
-----------(25360, 7328)


155

matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

...

/home/e/ezubc/c8y8/metaphor_detection/env/lib64/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    1.9s remaining:    5.8s
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.9s remaining:    3.2s
[Parallel(n_jobs=8)]: Done   4 out of   8 | elapsed:    1.9s remaining:    1.9s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.9s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   6 out of   8 | elapsed:    1.9s remaining:    0.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.9s finished
/home/e/ezubc/c8y8/metaphor_detection/env/lib64/python3.6/site-packages/scipy

...

Nulls: 2650 2650


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,178401,speak,speak,VVI,,,11759,kbh-fragment02,kbh,conversation,speak,-1,kbh-fragment02,Statement,-1.0
1,178407,speak,speak,VVI,,,11761,kbh-fragment02,kbh,conversation,speak,-1,kbh-fragment02,Statement,-1.0
2,178411,very,very,AV0,,,11761,kbh-fragment02,kbh,conversation,very,-1,kbh-fragment02,_,-1.0
3,178412,relaxed,relaxed,AJ0,,,11761,kbh-fragment02,kbh,conversation,relaxed,-1,kbh-fragment02,_,-1.0
4,178416,like,like,VVI,,,11762,kbh-fragment02,kbh,conversation,like,-1,kbh-fragment02,Similarity,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2645,193888,giving,give,VVG,,,14258,kbw-fragment17,kbw,conversation,giving,-1,kbw-fragment17,Giving,-1.0
2646,193895,got,get,VVN,,,14260,kbw-fragment17,kbw,conversation,got,-1,kbw-fragment17,Transition_to_state,-1.0
2647,193905,feel,feel,VVI,,,14261,kbw-fragment17,kbw,conversation,feel,-1,kbw-fragment17,_,-1.0
2648,193908,despised,despise,VVN,,,14261,kbw-fragment17,kbw,conversation,despised,-1,kbw-fragment17,_,-1.0


2650 2650


,word_id,word,lemma_x,word_type,function,seg_type,sentence_id,text_id,text_tag,genre,lemma_y,ac
0,178401,speak,speak,VVI,,,11759,kbh-fragment02,kbh,conversation,speak,3.70
1,178407,speak,speak,VVI,,,11761,kbh-fragment02,kbh,conversation,speak,3.70
2,178411,very,very,AV0,,,11761,kbh-fragment02,kbh,conversation,very,1.43
3,178412,relaxed,relaxed,AJ0,,,11761,kbh-fragment02,kbh,conversation,relaxed,2.50
4,178416,like,like,VVI,,,11762,kbh-fragment02,kbh,conversation,like,1.89
...,...,...,...,...,...,...,...,...,...,...,...,...
2645,193888,giving,give,VVG,,,14258,kbw-fragment17,kbw,conversation,give,2.83
2646,193895,got,get,VVN,,,14260,kbw-fragment17,kbw,conversation,get,2.38
2647,193905,feel,feel,VVI,,,14261,kbw-fragment17,kbw,conversation,feel,2.28
2648,193908,despised,despise,VVN,,,14261,kbw-fragment17,kbw,conversation,despise,2.12


2650 2650
(2650, 2921)
{'False': {'precision': 0.9374588545095458, 'recall': 0.6013513513513513, 'f1-score': 0.7326987393877027, 'support': 2368}, 'True': {'precision': 0.16534040671971706, 'recall': 0.6631205673758865, 'f1-score': 0.2646850672328379, 'support': 282}, 'accuracy': 0.6079245283018868, 'macro avg': {'precision': 0.5513996306146314, 'recall': 0.6322359593636189, 'f1-score': 0.4986919033102703, 'support': 2650}, 'weighted avg': {'precision': 0.8552937970466282, 'recall': 0.6079245283018868, 'f1-score': 0.6828950203131094, 'support': 2650}}


In [5]:
from ast import literal_eval

for file in list(glob.glob('results/*')):                                       
    with open(file, 'r') as f:
        print(file)
        d = literal_eval(f.readline())
        for metric in ['precision','recall', 'f1-score']:
            print(metric, d['weighted avg'][metric])
        print('\n')

results/U_academic.txt
precision 0.6512259226976289
recall 0.45444006492000927
f1-score 0.4971530738330662


results/GWC_academic.txt
precision 0.691677464054512
recall 0.5886853698121957
f1-score 0.6224652313961172


results/GT_fiction.txt
precision 0.7423149288099777
recall 0.728652186849152
f1-score 0.735273350099038


results/test_academic.txt
precision 0.691677464054512
recall 0.5886853698121957
f1-score 0.6224652313961172


results/LCGT_newspapers.txt
precision 0.6942077497702164
recall 0.632317562149158
f1-score 0.6575287248131031


results/AC_academic.txt
precision 0.7494485808062461
recall 0.5151866450266636
f1-score 0.5489016393755344


results/AC_newspapers.txt
precision 0.7375161482329508
recall 0.5322774659182037
f1-score 0.5756932927089282


results/LCGT_fiction.txt
precision 0.7431073525418317
recall 0.7349003272835466
f1-score 0.7389259562492985


results/GWC_newspapers.txt
precision 0.7169453301694374
recall 0.6309141940657578
f1-score 0.6618145806281146


results/GWC_